# Programming Project #2: Image Quilting

## CS445: Computational Photography - Fall 2019

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import utils
import os
import pdb
import random

In [2]:
from utils import cut # default cut function for seam finding section

### Part I: Randomly Sampled Texture (10 pts)

In [ ]:
sample_img_dir = 'data/pattern1.jpg' # feel free to change
sample_img = None
if os.path.exists(sample_img_dir):
    sample_img = cv2.imread(sample_img_dir)
    sample_img = cv2.cvtColor(sample_img,cv2.COLOR_BGR2RGB)
    plt.imshow(sample_img)

In [ ]:
def quilt_random(sample, out_size, patch_size):
    """
    Randomly samples square patches of size patchsize from sample in order to create an output image of size outsize.

    :param sample: numpy.ndarray   The image you read from sample directory
    :param out_size: int            The width of the square output image
    :param patch_size: int          The width of the square sample patch
    :return: numpy.ndarray
    """
    # TODO
    # init output image -> 2D matrix of dim out_size x out_size
    out_img = np.zeros(shape=(out_size,out_size,3))
    
    i = 0
    while i < out_size:
        # if bottom doesnt have enough room
        if i+patch_size>out_size:
            break
        j = 0
        while j < out_size:
            # if horizontal doesnt have enough room
            if j+patch_size>out_size:
                break
            # randomly sample patches from input
            input_row_size, input_col_size = sample.shape[0], sample.shape[1]
            y1 = np.random.randint(low=0,high=input_row_size-patch_size,size=1)[0]
            y2 = y1 + patch_size
            x1 = np.random.randint(low=0,high=input_col_size-patch_size,size=1)[0]
            x2 = x1 + patch_size
            sample_patch = sample[y1:y2,x1:x2]
            
            # stamp sample down on output img
            out_img[i:i+patch_size,j:j+patch_size] = sample_patch
            j+=patch_size
        i+=patch_size
    return out_img

In [ ]:
out_size = 700  # feel free to change to debug
patch_size = 100# feel free to change to debug

res = quilt_random(sample_img, out_size, patch_size)
# if res:
%matplotlib inline
plt.imshow(res/255)

### Part II: Overlapping Patches (30 pts)

In [ ]:
# ssd_patch performs template matching with the overlapping region, computing the cost of sampling each patch, 
# based on the sum of squared differences (SSD) of the overlapping regions of the existing and sampled patch.
# M = mask of current focus in image, I = image patch
def ssd_patch(sample, M, I):
    sample_size = len(sample)
    cost_image = np.zeros((sample_size, sample_size),dtype=np.float)
    im_R, im_G, im_B = I[:,:,0], I[:,:,1], I[:,:,2]
    samp_R, samp_G, samp_B = sample[:,:,0],sample[:,:,1],sample[:,:,2]
    ssd_R = ((M*im_R)**2).sum() - 2 * cv2.filter2D(samp_R, ddepth=-1, kernel = M*im_R) + cv2.filter2D(samp_R ** 2, ddepth=-1, kernel=M)
    ssd_G = ((M*im_G)**2).sum() - 2 * cv2.filter2D(samp_G, ddepth=-1, kernel = M*im_G) + cv2.filter2D(samp_G ** 2, ddepth=-1, kernel=M)
    ssd_B = ((M*im_B)**2).sum() - 2 * cv2.filter2D(samp_B, ddepth=-1, kernel = M*im_B) + cv2.filter2D(samp_B ** 2, ddepth=-1, kernel=M)
    cost_image = ssd_R + ssd_G + ssd_B

#     pdb.set_trace()
    # return array with cost for every pixel
#     print(cost_image)
    return cost_image

# choose_sample should take as input a cost image 
# (each pixel's value is the cost of selecting the patch centered at that pixel)
# and select a randomly sampled patch with low cost
def choose_sample(cost_image, sample, patch_size, tol):
    # randomly selects patch center with low cost
    # find smallest val in cost_image
    
    mid = int((patch_size-1)/2)
    small_cost_value = 0.1
    cost_image = cost_image[mid:len(cost_image)-mid,mid:len(cost_image)-mid]
    minc = np.amin(cost_image)
    minc=max(minc,small_cost_value)
    
    min_rows, min_cols = np.where( cost_image < minc*(1+tol))
    pdb.set_trace()
    rand_idx = random.randint(0,len(min_rows)-1)
    rand_row, rand_col = min_rows[rand_idx], min_cols[rand_idx]
    rand_row = rand_row + mid
    rand_col = rand_col + mid
    # pdb.set_trace()
    return sample[rand_row-mid:rand_row+mid+1, rand_col-mid:rand_col+mid+1]

In [ ]:
def quilt_simple(sample, out_size, patch_size, overlap, tol):
    """
    Randomly samples square patches of size patchsize from sample in order to create an output image of size outsize.
    Feel free to add function parameters
    :param sample: numpy.ndarray
    :param out_size: int
    :param patch_size: int
    :param overlap: int
    :param tol: int
    :return: numpy.ndarray
    """
    
    row = 0
    col = 0
    # create big image
    image = np.zeros((out_size, out_size,3),dtype=np.float64)
    #pdb.set_trace()
    
    # loop through big image
    while row + patch_size < out_size:
        print(str(row))
        while col +patch_size < out_size:
            #print(str(row)+" "+str(col))
            # create mask
            #   mask 0      mask 1    mask2    mask3
                # 0 0 0     1 0 0     1 1 1    1 1 1 
                # 0 0 0     1 0 0     0 0 0    1 0 0 
                # 0 0 0     1 0 0     0 0 0    1 0 0
            mask = np.zeros((patch_size, patch_size), dtype=np.float64)
            #mask 0
            if row == 0 and col == 0:
                # do nothing
                pass
            #mask 1
            elif row == 0:
                mask[:,:overlap] = 1
            #mask 2
            elif col == 0:
                mask[:overlap,:] = 1
            #mask 3
            else:
                mask[:overlap,:] = 1
                mask[:,:overlap] = 1
            #pdb.set_trace()
            
            # initialize mask that is patch_size depending on position
            image_temp = image[row:row+patch_size,col:col+patch_size]
            #pdb.set_trace()
            # create cost image using SSD
            cost_image = ssd_patch(sample, mask, image_temp)
            # pdb.set_trace()
            # choose random patch that is low cost
            quilt_patch = choose_sample(cost_image,sample, patch_size, tol)
            # add the quilt to the big image
            try:
                image[row:row+patch_size,col:col+patch_size] = quilt_patch
            except:
                pdb.set_trace()
            
            col += (patch_size - overlap)
            
        col = 0
        row += (patch_size - overlap)
    return image

In [ ]:
sample_img_dir = 'data/pattern1.jpg' # feel free to change
sample_img = None
if os.path.exists(sample_img_dir):
    sample_img = cv2.imread(sample_img_dir)
    sample_img = cv2.cvtColor(sample_img,cv2.COLOR_BGR2RGB)
    plt.imshow(sample_img)

In [ ]:
# out_size = 2000
# patch_size = 101
# overlap = 30
# tol = 0.0001
out_size = 750
patch_size = 121
overlap = 21
tol = 0.1

sample_img = np.array(sample_img, dtype='float64')
res = quilt_simple(sample_img, out_size, patch_size, overlap, tol) #feel free to change parameters to get best results
# if res:
#     plt.imshow(res)
%matplotlib inline
plt.imshow(res/255)

### Part III: Seam Finding (20 pts)


In [ ]:
# optional or use cut(err_patch) directly
def customized_cut(bndcost):
    pass

In [ ]:
# ssd_patch performs template matching with the overlapping region, computing the cost of sampling each patch, 
# based on the sum of squared differences (SSD) of the overlapping regions of the existing and sampled patch.
# M = mask of current focus in image, I = image patch
def ssd_patch(sample, M, I):
    sample_size = len(sample)
    cost_image = np.zeros((sample_size, sample_size),dtype=np.float)
    im_R, im_G, im_B = I[:,:,0], I[:,:,1], I[:,:,2]
    samp_R, samp_G, samp_B = sample[:,:,0],sample[:,:,1],sample[:,:,2]
    ssd_R = ((M*im_R)**2).sum() - 2 * cv2.filter2D(samp_R, ddepth=-1, kernel = M*im_R) + cv2.filter2D(samp_R ** 2, ddepth=-1, kernel=M)
    ssd_G = ((M*im_G)**2).sum() - 2 * cv2.filter2D(samp_G, ddepth=-1, kernel = M*im_G) + cv2.filter2D(samp_G ** 2, ddepth=-1, kernel=M)
    ssd_B = ((M*im_B)**2).sum() - 2 * cv2.filter2D(samp_B, ddepth=-1, kernel = M*im_B) + cv2.filter2D(samp_B ** 2, ddepth=-1, kernel=M)
    cost_image = ssd_R + ssd_G + ssd_B
    # return array with cost for every pixel
    return cost_image

# choose_sample should take as input a cost image 
# (each pixel's value is the cost of selecting the patch centered at that pixel)
# and select a randomly sampled patch with low cost
def choose_sample(cost_image, sample, patch_size, tol):
    # randomly selects patch center with low cost
    # find smallest val in cost_image
    
    mid = int((patch_size-1)/2)
    small_cost_value = 0.01
    cost_image = cost_image[mid:len(cost_image)-mid,mid:len(cost_image)-mid]
    minc = np.amin(cost_image)
    minc=max(minc,small_cost_value)
    
    min_rows, min_cols = np.where( cost_image < minc*(1+tol))
    random.seed(30)
    rand_idx = random.randint(0,len(min_rows)-1)
    rand_row, rand_col = min_rows[rand_idx], min_cols[rand_idx]
    rand_row = rand_row + mid
    rand_col = rand_col + mid
    # pdb.set_trace()
    return sample[rand_row-mid:rand_row+mid+1, rand_col-mid:rand_col+mid+1]

In [ ]:
#finds the min-cost contiguous path from the left to right side of 
#the patch according to the cost indicated by bndcost.
def quilt_cut(sample, out_size, patch_size, overlap, tol):
    """
    Samples square patches of size patchsize from sample using seam finding in order to create an output image of size outsize.
    Feel free to add function parameters
    :param sample: numpy.ndarray
    :param out_size: int
    :param patch_size: int
    :param overlap: int
    :param tol: int
    :return: numpy.ndarray
    """
    row = 0
    col = 0
    # create big image
    image = np.zeros((out_size, out_size,3),dtype=np.float64)
    mask_type = -1
    
    # loop through big image
    while row + patch_size < out_size:
        print(str(row))
        while col + patch_size < out_size:
            # create mask
            #   mask 0      mask 1    mask2    mask3
                # 0 0 0     1 0 0     1 1 1    1 1 1
                # 0 0 0     1 0 0     0 0 0    1 0 0 
                # 0 0 0     1 0 0     0 0 0    1 0 0
            mask = np.zeros((patch_size, patch_size), dtype=np.float64)
            #mask 0
            if row == 0 and col == 0:
                # do nothing
                pass
            #mask 1
            elif row == 0:
                mask[:,:overlap] = 1
            #mask 2
            elif col == 0:
                mask[:overlap,:] = 1
            #mask 3
            else:
                mask[:overlap,:] = 1
                mask[:,:overlap] = 1
            
            # initialize mask that is patch_size depending on position
            image_temp = image[row:row+patch_size,col:col+patch_size]
            # create cost image using SSD
            cost_image = ssd_patch(sample, mask, image_temp)
            
            # choose random patch that is low cost
            quilt_patch = choose_sample(cost_image,sample, patch_size, tol)
            #pdb.set_trace()
            
            # use cut to create a mask with cut boundary
            res = np.ones((patch_size, patch_size), dtype=np.float64)
            
            if row == 0 and col == 0:
                image[:patch_size,:patch_size] = quilt_patch
            elif row == 0:
                # take abs value and make it 2D instead of 3D
                pass_in_3D = image[row:row+patch_size, col:col+overlap] - quilt_patch[:, :overlap]
                pass_in_2D = pass_in_3D[:,:,0]**2 + pass_in_3D[:,:,1]**2 + pass_in_3D[:,:,2]**2
                cut_vert = cut(pass_in_2D.T).T
                quilt_R, quilt_G, quilt_B = quilt_patch[:,:,0], quilt_patch[:,:,1], quilt_patch[:,:,2]
                
#                 %matplotlib notebook
#                 plt.imshow(quilt_patch[:,:overlap]/255)
#                 plt.imshow(image[row:row+patch_size,col:col+overlap]/255)
#                 return
                
#                 b = quilt_patch[:,:overlap]
#                 c = image[row:row+patch_size,col:col+overlap]
#                 #where c and b are the template and patch
#                 dif = np.sum((c-b)**2, axis =2)
#                 d = cut_vert
                
#                 max = np.amax(dif)
#                 prev = 0
#                 for i in range(d.shape[0]):
#                     for j in range(d.shape[1]):
#                         print(dif[i, j])
#                         if prev!=d[i, j]:
#                             dif[i, j] = max
#                         prev = d[i, j]
                
#                 %matplotlib notebook
#                 plt.imshow(dif/255)
#                 break
                
                # iterate through image patch
                for x in range(row,row+patch_size):
                    for y in range(col,col+patch_size):
                        #pdb.set_trace()
                        if y >= col+overlap:
                            image[x,y] = quilt_patch[x-row,y-col]
                        else:
                            if cut_vert[x-row,y-col] != 0:
#                             if y > int((overlap-1)/2):
                                image[x,y] = quilt_patch[x-row,y-col]  
#                 %matplotlib notebook
#                 plt.imshow(image[row:row+patch_size,col:col+overlap]/255)
#                 return
            elif col == 0:
                pass_in_3D = image[row:row+overlap, col:col+patch_size] - quilt_patch[:overlap,:]
                pass_in_2D = pass_in_3D[:,:,0]**2 + pass_in_3D[:,:,1]**2 + pass_in_3D[:,:,2]**2
                cut_hor = cut(pass_in_2D)
                res[:overlap,:] = cut_hor
                
                quilt_R, quilt_G, quilt_B = quilt_patch[:,:,0], quilt_patch[:,:,1], quilt_patch[:,:,2]
                
                for x in range(row,row+patch_size):
                    for y in range(col,col+patch_size):
                        if y >= col+overlap:
                            image[x,y] = quilt_patch[x-row,y-col]
                        else:
                            if res[x-row,y-col] != 0:
                                image[x,y] = quilt_patch[x-row,y-col]
            else:
                #vert
                pass_in_3D_v = image[row:row+patch_size, col:col+overlap] - quilt_patch[:patch_size, :overlap]
                pass_in_2D_v = pass_in_3D_v[:,:,0]**2 + pass_in_3D_v[:,:,1]**2 + pass_in_3D_v[:,:,2]**2
                
                #hori
                pass_in_3D_h = image[row:row+overlap, col:col+patch_size] - quilt_patch[:overlap, :patch_size]
                pass_in_2D_h = pass_in_3D_h[:,:,0]**2 + pass_in_3D_h[:,:,1]**2 + pass_in_3D_h[:,:,2]**2
                #pass_in_h = image[row:row+overlap, col:col+patch_size] - quilt_patch[:overlap, :patch_size]
                cut_hor = cut(pass_in_2D_h)
                cut_vert = np.transpose(cut(np.transpose(pass_in_2D_v, (1,0))), (1,0))
                res[:overlap,:] = cut_hor
                res[:,:overlap] = res[:,:overlap]*cut_vert
                
                quilt_R, quilt_G, quilt_B = quilt_patch[:,:,0], quilt_patch[:,:,1], quilt_patch[:,:,2]
                
                # iterate through image patch
                for x in range(row,row+patch_size):
                    for y in range(col,col+patch_size):
                        #pdb.set_trace()
                        if y >= col+overlap:
                            image[x,y] = quilt_patch[x-row,y-col]
                        else:
                            if res[x-row,y-col] != 0:
                                image[x,y] = quilt_patch[x-row,y-col]
            col += (patch_size - overlap)
        col = 0
        row += (patch_size - overlap)
    return image

In [ ]:
sample_img_dir = 'data/pattern1.jpg' # feel free to change
sample_img = None
if os.path.exists(sample_img_dir):
    sample_img = cv2.imread(sample_img_dir)
    sample_img = cv2.cvtColor(sample_img,cv2.COLOR_BGR2RGB)
    plt.imshow(sample_img)

In [ ]:
# sample = small sample_img
out_size = 900
patch_size = 211
overlap = 91
tol = 0.0001
# out_size = 5000
# patch_size = 777
# overlap = 200
# tol = 0.0001

sample_img = np.array(sample_img, dtype='float64')
res = quilt_cut(sample_img, out_size, patch_size, overlap, tol) #feel free to change parameters to get best results
# if res:
#     plt.imshow(res)
%matplotlib inline
plt.imshow(res/255)

### part IV: Texture Transfer (30 pts)

In [ ]:
# ssd_patch performs template matching with the overlapping region, computing the cost of sampling each patch, 
# based on the sum of squared differences (SSD) of the overlapping regions of the existing and sampled patch.
# M = mask of current focus in image, I = image patch
def ssd_patch(sample, M, I):
    sample_size = len(sample)
    cost_image = np.zeros((sample_size, sample_size),dtype=np.float)
    im_R, im_G, im_B = I[:,:,0], I[:,:,1], I[:,:,2]
    samp_R, samp_G, samp_B = sample[:,:,0],sample[:,:,1],sample[:,:,2]
    ssd_R = ((M*im_R)**2).sum() - 2 * cv2.filter2D(samp_R, ddepth=-1, kernel = M*im_R) + cv2.filter2D(samp_R ** 2, ddepth=-1, kernel=M)
    ssd_G = ((M*im_G)**2).sum() - 2 * cv2.filter2D(samp_G, ddepth=-1, kernel = M*im_G) + cv2.filter2D(samp_G ** 2, ddepth=-1, kernel=M)
    ssd_B = ((M*im_B)**2).sum() - 2 * cv2.filter2D(samp_B, ddepth=-1, kernel = M*im_B) + cv2.filter2D(samp_B ** 2, ddepth=-1, kernel=M)
    cost_image = ssd_R + ssd_G + ssd_B
    # return array with cost for every pixel
    return cost_image

# choose_sample should take as input a cost image 
# (each pixel's value is the cost of selecting the patch centered at that pixel)
# and select a randomly sampled patch with low cost
def choose_sample(cost_image, sample, patch_size, tol):
    # randomly selects patch center with low cost
    # find smallest val in cost_image
    
    mid = int((patch_size-1)/2)
    small_cost_value = 0.01
    cost_image = cost_image[mid:len(cost_image)-mid,mid:len(cost_image)-mid]
    minc = np.amin(cost_image)
    minc=max(minc,small_cost_value)
    
    min_rows, min_cols = np.where( cost_image < minc*(1+tol))
    rand_idx = random.randint(0,len(min_rows)-1)
    rand_row, rand_col = min_rows[rand_idx], min_cols[rand_idx]
    rand_row = rand_row + mid
    rand_col = rand_col + mid
    # pdb.set_trace()
    return sample[rand_row-mid:rand_row+mid+1, rand_col-mid:rand_col+mid+1]

In [ ]:
def texture_transfer(sample, target, patch_size, overlap, tol):
    """
    Feel free to add function parameters
    """
    row = 0
    col = 0
    out_size = target.shape[0]
    # create big image
    image = np.zeros((out_size, out_size,3),dtype=np.float64)
    mask_type = -1
    
    # loop through big image
    while row + patch_size < out_size:
        print(str(row))
        while col + patch_size < out_size:
            # create mask
            #   mask 0      mask 1    mask2    mask3
                # 0 0 0     1 0 0     1 1 1    1 1 1
                # 0 0 0     1 0 0     0 0 0    1 0 0 
                # 0 0 0     1 0 0     0 0 0    1 0 0
            mask = np.zeros((patch_size, patch_size), dtype=np.float64)
            #mask 0
            if row == 0 and col == 0:
                # do nothing
                pass
            #mask 1
            elif row == 0:
                mask[:,:overlap] = 1
            #mask 2
            elif col == 0:
                mask[:overlap,:] = 1
            #mask 3
            else:
                mask[:overlap,:] = 1
                mask[:,:overlap] = 1
            
            # initialize mask that is patch_size depending on position
            image_temp = image[row:row+patch_size,col:col+patch_size]
            # create cost image using SSD
            cost_image = ssd_patch(sample, mask, image_temp) + ssd_patch(sample, np.ones((patch_size, patch_size)), target[row:row+patch_size, col:col+patch_size])
            
            # choose random patch that is low cost
            quilt_patch = choose_sample(cost_image,sample, patch_size, tol)
            #pdb.set_trace()
            
            # use cut to create a mask with cut boundary
            res = np.ones((patch_size, patch_size), dtype=np.float64)
            
            if row == 0 and col == 0:
                image[:patch_size,:patch_size] = quilt_patch
            elif row == 0:
                # take abs value and make it 2D instead of 3D
                pass_in_3D = image[row:row+patch_size, col:col+overlap] - quilt_patch[:, :overlap]
                pass_in_2D = pass_in_3D[:,:,0]**2 + pass_in_3D[:,:,1]**2 + pass_in_3D[:,:,2]**2
                cut_vert = cut(pass_in_2D.T).T
                quilt_R, quilt_G, quilt_B = quilt_patch[:,:,0], quilt_patch[:,:,1], quilt_patch[:,:,2]
                
                # iterate through image patch
                for x in range(row,row+patch_size):
                    for y in range(col,col+patch_size):
                        #pdb.set_trace()
                        if y >= col+overlap:
                            image[x,y] = quilt_patch[x-row,y-col]
                        else:
                            if res[x-row,y-col] != 0:
                                image[x,y] = quilt_patch[x-row,y-col]        
            elif col == 0:
                pass_in_3D = image[row:row+overlap, col:col+patch_size] - quilt_patch[:overlap,:]
                pass_in_2D = pass_in_3D[:,:,0]**2 + pass_in_3D[:,:,1]**2 + pass_in_3D[:,:,2]**2
                cut_hor = cut(pass_in_2D)
                res[:overlap,:] = cut_hor
                
                quilt_R, quilt_G, quilt_B = quilt_patch[:,:,0], quilt_patch[:,:,1], quilt_patch[:,:,2]
                
                for x in range(row,row+patch_size):
                    for y in range(col,col+patch_size):
                        if y >= col+overlap:
                            image[x,y] = quilt_patch[x-row,y-col]
                        else:
                            if res[x-row,y-col] != 0:
                                image[x,y] = quilt_patch[x-row,y-col]
            else:
                #vert
                pass_in_3D_v = image[row:row+patch_size, col:col+overlap] - quilt_patch[:patch_size, :overlap]
                pass_in_2D_v = pass_in_3D_v[:,:,0]**2 + pass_in_3D_v[:,:,1]**2 + pass_in_3D_v[:,:,2]**2
                
                #hori
                pass_in_3D_h = image[row:row+overlap, col:col+patch_size] - quilt_patch[:overlap, :patch_size]
                pass_in_2D_h = pass_in_3D_h[:,:,0]**2 + pass_in_3D_h[:,:,1]**2 + pass_in_3D_h[:,:,2]**2
                #pass_in_h = image[row:row+overlap, col:col+patch_size] - quilt_patch[:overlap, :patch_size]
                cut_hor = cut(pass_in_2D_h)
                cut_vert = np.transpose(cut(np.transpose(pass_in_2D_v, (1,0))), (1,0))
                res[:overlap,:] = cut_hor
                res[:,:overlap] = res[:,:overlap]*cut_vert
                
                quilt_R, quilt_G, quilt_B = quilt_patch[:,:,0], quilt_patch[:,:,1], quilt_patch[:,:,2]
                
                # iterate through image patch
                for x in range(row,row+patch_size):
                    for y in range(col,col+patch_size):
                        #pdb.set_trace()
                        if y >= col+overlap:
                            image[x,y] = quilt_patch[x-row,y-col]
                        else:
                            if res[x-row,y-col] != 0:
                                image[x,y] = quilt_patch[x-row,y-col]
            col += (patch_size - overlap)
        col = 0
        row += (patch_size - overlap)
    return image

In [ ]:
sample_img_dir = 'data/sketch.png' # feel free to change
target_img_dir = 'data/face.png'
sample_img = None
target_img = None
if os.path.exists(sample_img_dir):
    sample_img = cv2.imread(sample_img_dir)
    sample_img = cv2.cvtColor(sample_img,cv2.COLOR_BGR2RGB)
if os.path.exists(target_img_dir):
    target_img = cv2.imread(target_img_dir)
    target_img = cv2.cvtColor(target_img,cv2.COLOR_BGR2RGB)

In [ ]:
# sample = small sample_img
patch_size = 21
overlap = 5
tol = 0.0001

sample_img = np.array(sample_img, dtype='float64')
target_img = np.array(target_img, dtype='float64')

res = texture_transfer(sample_img, target_img, patch_size, overlap, tol)
#     plt.imshow(res)
%matplotlib inline
plt.imshow(res/255)

### Bells & Whistles

(10 pts) Create and use your own version of cut.m. To get these points, you should create your own implementation without basing it directly on the provided function (you're on the honor code for this one). 

You can simply copy your customized_cut(bndcost) into the box below so that it is easier for us to grade

(15 pts) Implement the iterative texture transfer method described in the paper. Compare to the non-iterative method for two examples.

(up to 20 pts) Use a combination of texture transfer and blending to create a face-in-toast image like the one on top. To get full points, you must use some type of blending, such as feathering or Laplacian pyramid blending.

(up to 40 pts) Extend your method to fill holes of arbitrary shape for image completion. In this case, patches are drawn from other parts of the target image. For the full 40 pts, you should implement a smart priority function (e.g., similar to Criminisi et al.).